In [1]:
import os, io, re, requests, json, pandas as pd

BASE = "https://data.cityofnewyork.us/resource"
EXPENSE_ID = "mwzb-yiwb"   # Expense Budget
ACA_ID     = "u42g-qjaj"   # Additional Costs Allocation (Pension/Fringe/Debt)
REVENUE_ID = "ugzk-a6x4"

def csv_query(dataset_id: str, soql: str) -> pd.DataFrame:
    url = f"{BASE}/{dataset_id}.csv"
    params = {"$query": soql}
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    return pd.read_csv(io.StringIO(r.text))

# Get data from API

In [2]:
FY = "2025"
PUBDATE = "20240630"

soql = f"""
SELECT agency_name, unit_appropriation_name, object_class_name, SUM(current_modified_budget_amount) AS total_amount
WHERE fiscal_year='{FY}' AND publication_date='{PUBDATE}'
GROUP BY agency_name, unit_appropriation_name, object_class_name
ORDER BY total_amount DESC
LIMIT 50000
"""

expenses = csv_query(EXPENSE_ID, soql)
expenses["total_amount"] = pd.to_numeric(expenses["total_amount"], errors="coerce")

In [3]:
FY = "2025"
PUBDATE_REVENUE = '2025 06 30'

soql = f"""
SELECT revenue_category_name, revenue_class_name, revenue_source_name, SUM(current_modified_budget_amount) AS total_amount
WHERE fiscal_year='{FY}' AND publication_date='{PUBDATE_REVENUE}'
GROUP BY revenue_category_name, revenue_class_name, revenue_source_name
ORDER BY total_amount DESC
LIMIT 50000
"""

revenue_orig = csv_query(REVENUE_ID, soql)
revenue_orig["total_amount"] = pd.to_numeric(revenue_orig["total_amount"], errors="coerce")

In [4]:
ACA_FY = "2025"
ACA_PUBDT = "20250501"
ACA_CAT = "Fringe Benefits"   # other options: "Pension", "Debt Service"

soql = f"""
SELECT agy_nm, SUM(ttl_amt) AS total_kdollars
WHERE fisc_yr='{ACA_FY}' AND pub_dt='{ACA_PUBDT}' AND cst_cat='{ACA_CAT}'
GROUP BY agy_nm
ORDER BY total_kdollars DESC
LIMIT 50000
"""

fringe = csv_query(ACA_ID, soql)
fringe["total_kdollars"] = pd.to_numeric(fringe["total_kdollars"], errors="coerce")
fringe["total_dollars"] = fringe["total_kdollars"] * 1_000

In [5]:
ACA_CAT = "Pensions" 

soql = f"""
SELECT agy_nm, SUM(ttl_amt) AS total_kdollars
WHERE fisc_yr='{ACA_FY}' AND pub_dt='{ACA_PUBDT}' AND cst_cat='{ACA_CAT}'
GROUP BY agy_nm
ORDER BY total_kdollars DESC
LIMIT 50000
"""

pension = csv_query(ACA_ID, soql)
pension["total_kdollars"] = pd.to_numeric(pension["total_kdollars"], errors="coerce")
pension["total_dollars"] = pension["total_kdollars"] * 1_000

In [6]:
ACA_CAT = "Debt Service" 

soql = f"""
SELECT agy_nm, SUM(ttl_amt) AS total_kdollars
WHERE fisc_yr='{ACA_FY}' AND pub_dt='{ACA_PUBDT}' AND cst_cat='{ACA_CAT}'
GROUP BY agy_nm
ORDER BY total_kdollars DESC
LIMIT 50000
"""

debt_service = csv_query(ACA_ID, soql)
debt_service["total_kdollars"] = pd.to_numeric(debt_service["total_kdollars"], errors="coerce")
debt_service["total_dollars"] = debt_service["total_kdollars"] * 1_000

# Clean revenue data

In [7]:
rev = revenue_orig.copy()
rev['bn_dol'] = rev.total_amount / 1_000_000_000
rev = rev.drop(columns=['total_amount'], axis=1)
colnames = ['revenue_category_name', 'revenue_class_name', 'revenue_source_name']
for col in colnames:
    rev[col] = rev[col].str.upper()

In [8]:
rev.revenue_category_name = rev.revenue_category_name.str.replace("-CATEGORICAL", "")

In [9]:
mask = rev.revenue_category_name=='FINES AND FOREITURES'
rev.loc[mask, 'revenue_category_name'] = 'FINES'
rev.loc[mask, colnames[1:3]] = 'NA'

In [10]:
others = [
    'MISCELLANEOUS', 'NON-GOVERNMENTAL GRANTS', 'TRANSFERS FROM OTHER FUNDS',
    'LICENS. PERM. PRIV, FRANCHISES', 'INTEREST INCOME',
    'UNRESTRICTED FEDERAL AND STATE AID', 'DISALLOWANCE CAT. GRANTS'
]
mask = rev.revenue_category_name.isin(others)
rev.loc[mask, 'revenue_category_name'] = 'OTHER'
rev.loc[mask, colnames[1:3]] = 'NA'

In [11]:
# rev.loc[rev.revenue_category_name=="STATE GRANTS AND CONTRACTS", 'revenue_source_name'] = 'NA'

mask = (
    (rev.revenue_category_name=="STATE GRANTS AND CONTRACTS") & 
    ~(rev.revenue_class_name.isin([
        'EDUCATION', 
        'SOCIAL SERVICES', 
        'HEALTH',
        'TRANSPORTATION',
        'MENTAL HEALTH', 
        'CRIMINAL JUSTICE'
    ]))
)
rev.loc[mask, 'revenue_class_name'] = 'OTHER'
rev.loc[mask, 'revenue_source_name'] = 'NA'

In [12]:
# rev.loc[rev.revenue_category_name=="FEDERAL GRANTS AND CONTRACTS", 'revenue_source_name'] = 'NA'

mask = (
    (rev.revenue_category_name=="FEDERAL GRANTS AND CONTRACTS") & 
    ~(rev.revenue_class_name.isin([
        'HEALTH AND HUMAN SERVICES', 
        'HOUSING AND URBAN DEVELOPMENT', 
        'EDUCATION',
        'AGRICULTURE',
        'DEPARTMENT OF HOMELAND SECURITY', 
        'TREASURY', 
        'TRANSPORTATION',
        'LABOR'
    ]))
)
rev.loc[mask, 'revenue_class_name'] = 'OTHER'
rev.loc[mask, 'revenue_source_name'] = 'NA'

In [13]:
rev.loc[:, 'revenue_source_name'] = rev.revenue_source_name.str.replace(r"TANF[-\s]*", "TANF - ", regex=True)

In [14]:
rev.loc[rev.revenue_class_name=='GENERAL PROPERTY TAXES', 'revenue_class_name'] = 'PROPERTY TAXES'
rev.loc[rev.revenue_class_name=='GENERAL SALES TAX', 'revenue_class_name'] = 'SALES TAXES'

In [15]:
mask = (
    (rev.revenue_category_name=="TAXES") & 
    ~(rev.revenue_class_name=="INCOME TAXES")
)
rev.loc[mask, 'revenue_source_name'] = "NA"

mask = (
    (rev.revenue_category_name=="TAXES") & 
    ~(rev.revenue_class_name.isin([
        'PROPERTY TAXES', 
        'INCOME TAXES',
        'SALES TAXES'
    ]))
)
rev.loc[mask, 'revenue_class_name'] = "OTHER"

In [16]:
rev.loc[rev.revenue_source_name=='PERSONAL INCOME TAX', 'bn_dol'] = (
    rev.loc[rev.revenue_source_name.isin([
        'PERSONAL INCOME TAX', 
        'PERS INC TAX  CTY EMP NON-RES', 
        'REFUNDS OF PERSONAL INCOME TAX'
    ]), 'bn_dol'].sum()
)  
rev = rev.loc[~(rev.revenue_source_name=='PERS INC TAX  CTY EMP NON-RES')]
rev = rev.loc[~(rev.revenue_source_name=='REFUNDS OF PERSONAL INCOME TAX')]

In [17]:
rev.loc[rev.revenue_source_name=='GENERAL CORPORATION TAX', 'bn_dol'] = (
    rev.loc[rev.revenue_source_name.isin([
        'GENERAL CORPORATION TAX', 
        'REFUNDS OF GENERAL CORP TAX'
    ]), 'bn_dol'].sum()
)  
rev = rev.loc[~(rev.revenue_source_name=='REFUNDS OF GENERAL CORP TAX')]

In [18]:
rev.loc[rev.revenue_source_name.isin([
    "UNINCORPORATED BUSINESS INC TX",
    "REFUNDS OF UNICORP BUSN TAX", 
    "PASS THROUGH ENTITY TAX",
    "REFUNDS OF PASS THROUGH ENTITY TAX"
]), 'revenue_source_name'] = 'BUSINESS INCOME TAX'

In [19]:
rev.loc[rev.revenue_category_name=="CHARGES FOR SERVICES", 'revenue_source_name'] = 'NA'

In [20]:
rev = rev.groupby(colnames,as_index=False).agg(bn_dol=('bn_dol', 'sum'))

In [21]:
rev.loc[:, 'bn_dol'] = pd.to_numeric(rev.bn_dol, errors='coerce')
rev = rev[rev.bn_dol > 0.005]
rev.loc[:, 'bn_dol'] = round(rev.bn_dol, 2)

# Clean expenses data

In [22]:
BOROUGH_TO_COUNTY = {
    "BRONX": "BRONX COUNTY",
    "KINGS": "KINGS COUNTY",
    "NEW YORK": "NEW YORK COUNTY",
    "QUEENS": "QUEENS COUNTY",
    "RICHMOND": "RICHMOND COUNTY",
}

def normalize_agency(series: pd.Series) -> pd.Series:
    def norm(s):
        if pd.isna(s):
            return s
        s = str(s).upper().strip()

        # ---------- basic cleanup ----------
        s = s.replace("&", " AND ")
        s = re.sub(r"[.’']", "", s)                  # remove apostrophes/periods
        s = re.sub(r"\s+", " ", s).strip()

        # ---------- generic expansions ----------
        s = re.sub(r"\bDEPT\b", "DEPARTMENT", s)
        s = re.sub(r"\bSVCS?\b|\bSVS\b", "SERVICES", s)
        s = re.sub(r"\bCORP\b", "CORPORATION", s)

        # ---------- acronyms / short names ----------
        s = re.sub(r"\bHPD\b", "HOUSING PRESERVATION AND DEVELOPMENT", s)
        s = re.sub(r"\bDOITT\b", "DEPARTMENT OF INFORMATION TECHNOLOGY AND TELECOMMUNICATIONS", s)
        s = re.sub(r"\bDORIS\b", "DEPARTMENT OF RECORDS AND INFORMATION SERVICES", s)
        s = re.sub(r"\bFISA\b", "FINANCIAL INFORMATION SERVICE AGENCY", s)
        s = re.sub(r"\bHHC\b", "HEALTH AND HOSPITALS CORPORATION", s)
        s = re.sub(r"\bC\.?U\.?N\.?Y\.?\b|\bC U N Y\b", "CITY UNIVERSITY OF NEW YORK", s)

        # ---------- specific short → full ----------
        s = re.sub(r"^POLICE DEPT$", "POLICE DEPARTMENT", s)
        s = re.sub(r"^FIRE DEPT$", "FIRE DEPARTMENT", s)

        # Youth & Community Development truncations → full name
        s = re.sub(r"^(DEPARTMENT OF )?YOUTH AND COMMUNITY DVLPM?T$", 
           "DEPARTMENT OF YOUTH AND COMMUNITY DEVELOPMENT", s)
        s = re.sub(r"^DEPARTMENT OF YOUTH AND COMMUNITY DEV$", 
           "DEPARTMENT OF YOUTH AND COMMUNITY DEVELOPMENT", s)
        
        # Citywide Administrative Services (cover truncations)
        s = re.sub(r"^CITYWIDE ADMIN(ISTRATIVE)? SERV(ICE|ICES)?$",
                   "DEPARTMENT OF CITYWIDE ADMINISTRATIVE SERVICES", s)
        s = re.sub(r"^DEPARTMENT OF CITYWIDE ADMIN SERVICE$",
                   "DEPARTMENT OF CITYWIDE ADMINISTRATIVE SERVICES", s)

        # Environmental Protection (cover truncations/variants)
        s = re.sub(r"^(ENVIRONMENTAL PROTECTION|DEPARTMENT OF ENVIRONMENTAL PROTEC(T|TION)?\.?)$",
                   "DEPARTMENT OF ENVIRONMENTAL PROTECTION", s)

        # DoITT family (cover truncation)
        s = re.sub(r"^DEPARTMENT OF INFO TECH( NOLOGY)? AND TELECOMM(UNICATIONS)?$",
                   "DEPARTMENT OF INFORMATION TECHNOLOGY AND TELECOMMUNICATIONS", s)

        # Mental Health legacy → DOHMH
        s = re.sub(r"^DEPARTMENT OF MENTAL HEALTH$", "DEPARTMENT OF HEALTH AND MENTAL HYGIENE", s)

        # Consumer Affairs → Consumer & Worker Protection (historic rename)
        s = re.sub(r"^DEPARTMENT OF CONSUMER AFFAIRS$",
                   "DEPARTMENT OF CONSUMER AND WORKER PROTECTION", s)

        # Taxi & Limousine (cover “NYC … COMM”)
        s = re.sub(r"^(NYC )?TAXI AND LIMOUSINE COMM(ISSION)?$",
                   "TAXI AND LIMOUSINE COMMISSION", s)

        # OATH (accept AND/& and singular “HEARING”)
        s = re.sub(r"^OFFICE OF ADMIN(ISTRATIVE)? TRIALS ?(?:AND|&) ?HEARINGS?$",
                   "OFFICE OF ADMINISTRATIVE TRIALS AND HEARINGS", s)
        s = re.sub(r"^OFFICE OF ADMIN TRIALS AND HEARINGS$",
                   "OFFICE OF ADMINISTRATIVE TRIALS AND HEARINGS", s)

        # Children’s Services (incl “ADMIN FOR …”)
        s = re.sub(r"^ADMIN(ISTRATION)? FOR CHILDRENS? SERVICES$",
                   "ADMINISTRATION FOR CHILDRENS SERVICES", s)

        # Libraries
        s = re.sub(r"^RESEARCH LIBRARIES$", "NEW YORK RESEARCH LIBRARIES", s)

        # Planning / Landmarks / HR / Payroll / Collective Bargaining / EEPC
        s = re.sub(r"^CITY PLANNING$", "DEPARTMENT OF CITY PLANNING", s)
        s = re.sub(r"^LANDMARKS PRESERVATION( COMM)?$",
                   "LANDMARKS PRESERVATION COMMISSION", s)
        s = re.sub(r"^HUMAN RIGHTS COMMISSION$", "COMMISSION ON HUMAN RIGHTS", s)
        s = re.sub(r"^PAYROLL ADMINISTRATION$", "OFFICE OF PAYROLL ADMINISTRATION", s)
        s = re.sub(r"^COLLECTIVE BARGAINING$", "OFFICE OF COLLECTIVE BARGAINING", s)
        s = re.sub(r"^EQUAL EMPLOYMENT PRACTICES$",
                   "EQUAL EMPLOYMENT PRACTICES COMMISSION", s)

        # Comptroller / central categories
        s = re.sub(r"^COMPTROLLER$", "OFFICE OF THE COMPTROLLER", s)
        s = re.sub(r"^DEBT SERVICES?$", "DEBT SERVICE", s)
        s = re.sub(r"^PENSIONS?$", "PENSION CONTRIBUTIONS", s)
        s = re.sub(r"^MAC DEBT$", "MUNICIPAL ASSISTANCE CORPORATION DEBT", s)

        # Independent Budget Office
        s = re.sub(r"^INDEPENDENT BUDGET OFFICE(?: - IBO)?$",
                   "INDEPENDENT BUDGET OFFICE", s)

        # Tax Commission old label
        s = re.sub(r"^TAX COMMISSION$",
                   "OFFICE OF ADMINISTRATIVE TAX APPEALS", s)

        # ----- DA / BP / PA patterns -----
        m = re.match(r"^DA - (BRONX|KINGS|NEW YORK|QUEENS|RICHMOND)$", s)
        if m:
            s = f"DISTRICT ATTORNEY { BOROUGH_TO_COUNTY[m.group(1)] }"

        m = re.match(r"^BOROUGH PRESIDENT(?: -)? (BRONX|BROOKLYN|MANHATTAN|QUEENS|STATEN ISLAND)$", s)
        if m:
            s = f"BOROUGH PRESIDENT - {m.group(1)}"
        m = re.match(r"^BP - (BRONX|BROOKLYN|MANHATTAN|QUEENS|STATEN ISLAND)$", s)
        if m:
            s = f"BOROUGH PRESIDENT - {m.group(1)}"

        # Normalize PUBLIC ADMINISTRATOR dash spacing; map PA - BOROUGH
        s = re.sub(r"^PUBLIC ADMINISTRATOR\s*-\s*", "PUBLIC ADMINISTRATOR-", s)
        m = re.match(r"^PA - (KINGS|NEW YORK|QUEENS|BRONX|RICHMOND)$", s)
        if m:
            s = f"PUBLIC ADMINISTRATOR-{ BOROUGH_TO_COUNTY[m.group(1)] }"

        # Special Narcotics
        if s == "SPECIAL NARCOTICS":
            s = "OFFICE OF PROSECUTION SPEC NARCO"

        # Ensure single spaces around dashes
        s = re.sub(r"\s*-\s*", " - ", s)
        # Remove double spaces just in case
        s = re.sub(r"\s{2,}", " ", s).strip()

        return s

    return series.map(norm)


def normalize_unit(series: pd.Series) -> pd.Series:
    def norm(s):
        s = re.sub(r"\s*[-(]?\s*OTPS\s*[)-]?", " - VENDORS/EQUIPMENT", s)
        s = re.sub(r"\s*[-(]?\s*PS\s*[)-]?", " - EMPLOYEES", s)
        return s
    return series.map(norm)


def coalesce_community_boards(df: pd.DataFrame) -> pd.DataFrame:
    is_cb = df["agency_name"].str.contains(r"\bCOMMUNITY BOARD #\d+\b", regex=True, na=False)
    df.loc[is_cb, "agency_name"] = "COMMUNITY BOARDS"
    
    out = df.groupby(
        ["agency_name","unit_appropriation_name","object_class_name"], 
        as_index=False
    ).agg(total_amount=("total_amount","sum"))
    
    return out

In [23]:
expenses["agency_name"] = normalize_agency(expenses["agency_name"])
expenses = coalesce_community_boards(expenses)

drop_mask = (
    (
        (expenses.agency_name == "MISCELLANEOUS") & 
        (expenses.unit_appropriation_name == "FRINGE BENEFITS")
    ) | 
    (expenses.agency_name == "DEBT SERVICE") | 
    (expenses.agency_name == "PENSION CONTRIBUTIONS")
)
expenses = expenses[~drop_mask]

for df in (pension, fringe, debt_service):
    df.rename(columns={"agy_nm": "agency_name", "total_dollars": "total_amount"}, inplace=True)
    df['object_class_name'] = 'NA'
    df.drop('total_kdollars', axis=1, inplace=True)
    df["agency_name"] = normalize_agency(df["agency_name"])

pension['unit_appropriation_name'] = 'PENSION CONTRIBUTIONS (ACA)'
fringe['unit_appropriation_name'] = 'FRINGE BENEFITS (ACA)'
debt_service['unit_appropriation_name'] = 'DEBT SERVICE (ACA)'

combined = pd.concat([expenses, pension, fringe, debt_service], ignore_index=True)
combined['bn_dol'] = combined['total_amount'] / 1_000_000_000

In [24]:
# combined.to_csv("combined_data.csv")
# combined = read_csv("combined_data.csv")

In [25]:
top_10_agencies = combined.groupby(["agency_name"], as_index=False).agg(bn_dol = ("bn_dol", "sum")).sort_values(by="bn_dol", ascending=False).head(11).agency_name.values
top_10_but_not_misc = (combined.agency_name.isin(top_10_agencies)) & (combined.agency_name != "MISCELLANEOUS")
combined_top_10 = combined.loc[top_10_but_not_misc].copy()
combined_other = combined.loc[~top_10_but_not_misc].copy()

In [26]:
combined_top_10 = combined_top_10.drop(columns=['object_class_name', 'total_amount'], axis=1)
combined_top_10 = combined_top_10.rename(columns={"agency_name": "agency", "unit_appropriation_name": "detail"})
combined_top_10 = combined_top_10.groupby(["agency", "detail"], as_index=False).agg(bn_dol = ("bn_dol", "sum"))

In [27]:
combined_other['agency'] = "OTHER"
combined_other = combined_other.drop(columns=['object_class_name', 'total_amount', 'unit_appropriation_name'], axis=1)
combined_other = combined_other.rename(columns={"agency_name": "detail"})

top_10_other = combined_other.groupby(["detail"], as_index=False).agg(bn_dol = ("bn_dol", "sum")).sort_values(by="bn_dol", ascending=False).head(11).detail.values
top_10_other_but_not_misc = (combined_other.detail.isin(top_10_other)) & (combined_other.detail != "MISCELLANEOUS")
combined_other.loc[~top_10_other_but_not_misc, 'detail'] = 'REMAINDER INCLUDING MISCELLANEOUS'

combined_other = combined_other.groupby(["agency", "detail"], as_index=False).agg(bn_dol = ("bn_dol", "sum"))

In [28]:
combined_clean = pd.concat([combined_top_10, combined_other], ignore_index=True)
combined_clean.bn_dol = pd.to_numeric(combined_clean.bn_dol, errors='coerce')
combined_clean = combined_clean[combined_clean.bn_dol > 0.005]
combined_clean.bn_dol = round(combined_clean.bn_dol, 2)
combined_clean = combined_clean.sort_values(by='bn_dol', ascending=False)

In [29]:
combined_clean['detail'] = normalize_unit(combined_clean['detail'])

In [30]:
# combined_clean.to_csv("combined_clean_data.csv")

# Turn into a json 

In [31]:
def df_to_json(expenditure_df, revenue_df):
    # Initialize the root structure
    data = {
        "name": "NYC FY2025 BUDGET (BILLIONS OF DOLLARS)",
        "children": [
            {
                "name": "EXPENDITURE",
                "children": []
            },
            {
                "name": "REVENUE",
                "children": []
            }
        ]
    }
    
    # Process EXPENDITURE dataframe
    agencies = expenditure_df['agency'].unique()
    for agency in agencies:
        agency_data = expenditure_df[expenditure_df['agency'] == agency]
        details = [
            {
                "name": row['detail'],
                "value": row['bn_dol']
            } for _, row in agency_data.iterrows()
        ]
        # Add agency node to EXPENDITURE children
        data['children'][0]['children'].append({
            "name": agency,
            "children": details
        })
    
    # Process REVENUE dataframe
    categories = revenue_df['revenue_category_name'].unique()
    for category in categories:
        category_data = revenue_df[revenue_df['revenue_category_name'] == category]
        category_node = {"name": category}
        
        # Group by revenue_class_name
        classes = category_data['revenue_class_name'].unique()
        class_nodes = []
        for class_name in classes:
            class_data = category_data[category_data['revenue_class_name'] == class_name]
            
            # If revenue_class_name is "NA", assign value to category node
            if class_name == "NA":
                total_value = class_data['bn_dol'].sum()
                category_node["value"] = total_value
            else:
                class_node = {"name": class_name}
                source_nodes = []
                
                # Process revenue_source_name
                sources = class_data['revenue_source_name'].unique()
                for source in sources:
                    source_data = class_data[class_data['revenue_source_name'] == source]
                    if source == "NA":
                        # If source is "NA", assign value to class node
                        total_value = source_data['bn_dol'].sum()
                        class_node["value"] = total_value
                    else:
                        # Source is not "NA", add as leaf node
                        for _, row in source_data.iterrows():
                            source_nodes.append({
                                "name": row['revenue_source_name'],
                                "value": row['bn_dol']
                            })
                
                # Only add children to class node if there are source nodes
                if source_nodes:
                    class_node["children"] = source_nodes
                
                # Only append class node if it has children or a value
                if source_nodes or class_node.get('value') is not None:
                    class_nodes.append(class_node)
        
        # Only add children to category node if there are class nodes
        if class_nodes:
            category_node["children"] = class_nodes
        
        # Only append category node if it has children or a value
        if class_nodes or category_node.get('value') is not None:
            data['children'][1]['children'].append(category_node)
    
    # Remove empty children arrays from top-level nodes
    for child in data['children']:
        if not child['children']:
            del child['children']
    
    return data

In [32]:
combined_clean_json = df_to_json(combined_clean, rev)

In [33]:
with open('combined_clean_data_json.json', 'w') as f:
    json.dump(combined_clean_json, f, indent=4)